# Building Bicycle-Rider Models with BRiM

Welcome to this tutorial on creating **B**icycle-**Ri**der **M**odels using BRiM!

In this notebook, we'll embark on a hands-on journey into BRiM, focusing on the practical aspects from a model user's perspective. By the end of this tutorial, you'll have achieved the following learning goals:

- Create a Carvallo-Whipple bicycle model using BRiM.
- Create a custom bicycle model to suit specific needs.
- Extend a bicycle model to include a rider.
- Parameterize a model using the [BicycleParameters](https://github.com/moorepants/BicycleParameters) library.
- Simulate models using a simple `Simulator` utility.
- Visualize a model's behavior with ease using the integrated plotting utility powered by [SymMePlot](https://github.com/TJStienstra/symmeplot).

Before diving into this tutorial, it's recommended that you familiarize yourself with [the introduction](https://mechmotum.github.io/brim/guides/introduction.html) and read through the sections on _Software Overview_ to _BRiM Models_ in [the BRiM paper](https://doi.org/10.59490/6504c5a765e8118fc7b106c3). This background information will provide valuable context for what we'll cover here.

## Tutorial Overview

This tutorial is structured as follows: We'll begin by working with the default Carvallo-Whipple bicycle model, adhering to [Moore's convention](https://moorepants.github.io/dissertation/eom.html). Our journey will encompass configuring the model in BRiM, exporting it, and deriving the equations of motion (EoMs). Subsequently, we'll venture into configuring a new bicycle model, this time incorporating fork suspension. This model will be further extended to include an upper-body rider. Next, we'll parametrize these models, enabling us to conduct multiple simulations. Finally, we'll visualize our results as captivating animations.

## The Default Bicycle Model

Our starting point is the Carvallo-Whipple bicycle model, following [Moore's convention](https://moorepants.github.io/dissertation/eom.html). The diagram below illustrates the general configuration of this model:

<center><img width=800px src="https://moorepants.github.io/dissertation/_images/bicycle-coordinates.png"/></center>

For our initial model, we'll build the default Carvallo-Whipple bicycle, which aligns perfectly with the image above:

<center><img width=800px src="../_static/configuration_overview_whipple_default.svg" alt="Configuration overview Carvallo-Whipple bicycle SVG"/></center>

When configuring a bicycle in BRiM, each component must be assigned a unique name. Symbol names are derived from component names, so using distinct names is essential. For example, if you would provide `"wheel"` as the name for both the front wheel and the rear wheel then the symbol for the radius will also be the same symbol. Additionally, BRiM applies conventions for certain components. For instance, rear and front frames follow Moore's convention by default.

_Note: You can explicitly specify convention usage by employing `RigidRearFrame.from_convention("moore", "rear_frame")`._

**Exercise**: The code below initiates the bicycle configuration. Complete it to match the configuration displayed in the image above.


In [ ]:
import warnings
import brim as bm
import sympy as sm
import sympy.physics.mechanics as me

from brim.bicycle import WhippleBicycleMoore, RigidRearFrameMoore
from brim.rider import SphericalShoulderTorque, PinElbowTorque
from brim.brim import SideLeanSeatSpringDamper

In [ ]:
bicycle_def = bm.WhippleBicycle("my_bike")
assert type(bicycle_def) == WhippleBicycleMoore
bicycle_def.rear_frame = bm.RigidRearFrame.from_convention("moore", "rear_frame")
assert type(bicycle_def.rear_frame) == RigidRearFrameMoore
bicycle_def.rear_wheel = bm.KnifeEdgeWheel("rear_wheel")
bicycle_def.rear_tire = bm.NonHolonomicTire("rear_tire")
### BEGIN SOLUTION
bicycle_def.ground = bm.FlatGround("ground")
bicycle_def.front_frame = bm.RigidFrontFrame("front_frame")
bicycle_def.front_wheel = bm.KnifeEdgeWheel("front_wheel")
bicycle_def.front_tire = bm.NonHolonomicTire("front_tire")
### END SOLUTION
assert len(bicycle_def.submodels) == 5
assert len(bicycle_def.connections) == 2

With the model configured we can now let BRiM define the entire model using [define_all](https://mechmotum.github.io/brim/_autosummary/brim.core.base_classes.ModelBase.html#brim.core.base_classes.ModelBase.define_all). This method actually calls five other methods in order, of which the last four were also discussed in the four-bar linkage tutorial:

1. `define_connections`: All models associate the connections with their required submodels.
2. `define_objects`: Create the objects, such as symbols reference frames, without defining any relationships between them.
3. `define_kinematics`: Establish relationships between the objects' orientations/positions, velocities, and accelerations.
4. `define_loads`: Specifies the forces and torques acting upon the system.
5. `define_constraints`: Computes the holonomic and nonholonomic constraints to which the system is subject.

A reason why you may want to call each of these methods manually in order is that you would like to intervene. For example, you may want to change a symbol as done below.

In [ ]:
# Manually run all steps
bicycle_def.define_connections()
bicycle_def.define_objects()
print(bicycle_def.rear_wheel.radius, bicycle_def.front_wheel.radius)
# Change the symbol names of the rear and front wheel radius
bicycle_def.rear_wheel.symbols["r"] = sm.Symbol("Rr")
bicycle_def.front_wheel.symbols["r"] = sm.Symbol("Rf")
print(bicycle_def.rear_wheel.radius, bicycle_def.front_wheel.radius)
bicycle_def.define_kinematics()
bicycle_def.define_loads()
bicycle_def.define_constraints()

Now we can export this model to a `System` object from `sympy.physics.mechanics`.

In [ ]:
system_def = bicycle_def.to_system()

At the moment we have not specified any loads, not even gravity. The simplest way to apply gravity is to use `System.apply_uniform_gravity`. To disturb the bicycle we can apply a lateral force at the saddle, and to control the bicycle a torque actuator at the steer.

_Note: loads and constraints can almost always be applied after exporting a model to a system object. The only instance when this is not the case is when the kinematics have to be altered in any way, e.g. using auxiliary speeds._

In [ ]:
normal = bicycle_def.ground.get_normal(bicycle_def.ground.origin)
g = sm.symbols("g")
disturbance = me.dynamicsymbols("disturbance")
steer_torque = me.dynamicsymbols("steer_torque")
system_def.apply_uniform_gravity(-g * normal)
system_def.add_loads(
    me.Force(bicycle_def.rear_frame.saddle.point, disturbance * bicycle_def.rear_frame.wheel_hub.axis)
)
system_def.add_actuators(
    me.TorqueActuator(steer_torque, bicycle_def.rear_frame.steer_hub.axis,
                      bicycle_def.rear_frame.steer_hub.frame, bicycle_def.front_frame.steer_hub.frame)
)
system_def.loads, system_def.actuators

Before forming the EoMs we need to specify which generalized coordinates and speeds are independent and which are dependent.

In [ ]:
try:
    system_def.validate_system()
except ValueError as e:
    print(e)
system_def.q_ind = [*bicycle_def.q[:4], *bicycle_def.q[5:]]
system_def.q_dep = [bicycle_def.q[4]]
system_def.u_ind = [bicycle_def.u[3], *bicycle_def.u[5:7]]
system_def.u_dep = [*bicycle_def.u[:3], bicycle_def.u[4], bicycle_def.u[7]]
system_def.validate_system()

Now that everything is define, we can form the EoMs (you can ignore the warnings).

_Note: The reason to use `"CRAMER"` as constraint solver is to prevent zero-divisions that otherwise occur._

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    eoms = system_def.form_eoms(constraint_solver="CRAMER")

Congratulations you've just created your first bicycle model using BRiM!

## Modifying a Bicycle
Now that you've got a handle on creating a basic bicycle model, let's start modifying it to include some fork suspension. The best method is to create a new `WhippleBicycle` object to prevent possible clashes. The model we will use is [SuspensionRigidFrontFrame](https://mechmotum.github.io/brim/_autosummary/brim.bicycle.front_frames.SuspensionRigidFrontFrame.html).

**Exercise**: Complete the configuration of the bicycle model with fork suspension below.

In [ ]:
bicycle = bm.WhippleBicycle("bicycle")
### BEGIN SOLUTION
bicycle.ground = bm.FlatGround("ground")
bicycle.rear_wheel = bm.KnifeEdgeWheel("rear_wheel")
bicycle.rear_frame = bm.RigidRearFrame("rear_frame")
bicycle.front_frame = bm.SuspensionRigidFrontFrame("front_frame")
bicycle.front_wheel = bm.KnifeEdgeWheel("front_wheel")
bicycle.rear_tire = bm.NonHolonomicTire("rear_tire")
bicycle.front_tire = bm.NonHolonomicTire("front_tire")
### END SOLUTION
assert len(bicycle.submodels) == 5
assert len(bicycle.connections) == 2
assert isinstance(bicycle.front_frame, bm.SuspensionRigidFrontFrame)

In theory, you could follow the same process now as before and then you will have formed the EoMs of your (first) custom bicycle model. However, we'll continue by using this bicycle model in a bicycle-rider model.

## Extending with a Rider
We can also extend the model with an upper-body rider. We will attach a side-leaning upper-body rider model, where the shoulders are modeled as spherical joints and the elbows as pin joints. To make this extension we create instantiate a [bicycle-rider model](https://mechmotum.github.io/brim/_autosummary/brim.brim.bicycle_rider.BicycleRider.html). This instance we'll have to provide both our bicycle model, rider model, and the method to connect them. The image below shows the constitution of a complete rider model (not the one we are modeling currently).

<center><img src="../_static/configuration_overview_rider.svg" alt="Configuration overview of a rider SVG"/></center>

Let us start with the rider model.


In [ ]:
rider = bm.Rider("rider")
rider.pelvis = bm.PlanarPelvis("pelvis")
rider.torso = bm.PlanarTorso("torso")
rider.left_arm = bm.PinElbowStickLeftArm("left_arm")
rider.right_arm = bm.PinElbowStickRightArm("right_arm")
rider.sacrum = bm.FixedSacrum("sacrum")
rider.left_shoulder = bm.FlexRotLeftShoulder("left_shoulder")
rider.right_shoulder = bm.FlexRotRightShoulder("right_shoulder")

Now let us combine the bicycle and rider models into a single bicycle-rider model. The image below shows the constitution of a complete bicycle-rider model (not the one we are modeling currently). We will just be adding the two models as submodels and the [FixedSeat](https://mechmotum.github.io/brim/_autosummary/brim.brim.seats.FixedSeat.html) and [HolonomicHandGrips](https://mechmotum.github.io/brim/_autosummary/brim.brim.hand_grips.HolonomicHandGrips.html) connections.

<center><img src="../_static/configuration_overview_bicycle_rider.svg" alt="Configuration overview of a bicycle-rider SVG"/></center>

**Exercise**: Configure the bicycle-rider model accordingly.

In [ ]:
bicycle_rider = bm.BicycleRider("bicycle_rider")
### BEGIN SOLUTION
bicycle_rider.bicycle = bicycle
bicycle_rider.rider = rider
bicycle_rider.seat = bm.FixedSeat("seat")
bicycle_rider.hand_grips = bm.HolonomicHandGrips("hand_grips")
### END SOLUTION
assert len(bicycle_rider.submodels) == 2
assert len(bicycle_rider.connections) == 2

To control the bicycle we can make the rider torque-driven by applying torques with the elbows. For this purpose BRiM has already dedicated load groups, e.g. [PinElbowTorque](https://mechmotum.github.io/brim/_autosummary/brim.rider.arms.PinElbowTorque.html).

**Exercise**: Instantiate a load group for each arm with a unique name and assign them to the variables `left_arm_lg` and `right_arm_lg`. Next, add each load group to the corresponding arm using [add_load_groups](https://mechmotum.github.io/brim/_autosummary/brim.rider.arms.ArmBase.html#brim.rider.arms.ArmBase.add_load_groups).

In [ ]:
### BEGIN SOLUTION
left_arm_lg = PinElbowTorque("left_elbow")
rider.left_arm.add_load_groups(left_arm_lg)
right_arm_lg = PinElbowTorque("right_elbow")
rider.right_arm.add_load_groups(right_arm_lg)
### END SOLUTION
assert isinstance(left_arm_lg, PinElbowTorque)
assert isinstance(right_arm_lg, PinElbowTorque)
assert len(rider.left_arm.load_groups) == 1
assert len(rider.right_arm.load_groups) == 1

With the model configured we can let BRiM define the model.

_Note: to simplify the EoMs we can set the yaw and roll angles of the rider with respect to the rear frame to zero._

In [ ]:
bicycle_rider.define_connections()
bicycle_rider.define_objects()
bicycle_rider.seat.symbols["yaw"] = 0
bicycle_rider.seat.symbols["roll"] = 0
bicycle_rider.define_kinematics()
bicycle_rider.define_loads()
bicycle_rider.define_constraints()

**Exercise**: Export the model to single `System` instance and assign it to the variable `system_br` and apply a gravitational and a disturbance force like before.

In [ ]:
### BEGIN SOLUTION
system_br = bicycle_rider.to_system()
normal = bicycle_rider.bicycle.ground.get_normal(bicycle_rider.bicycle.ground.origin)
g = sm.symbols("g")
system_br.apply_uniform_gravity(-g * normal)
system_br.add_loads(
    me.Force(bicycle_rider.bicycle.rear_frame.saddle.point,
             disturbance * bicycle_rider.bicycle.rear_frame.wheel_hub.axis)
)
### END SOLUTION
system_br.loads, system_br.actuators

**Exercise**: Set the independent and dependent generalized coordinates and speeds of `system_br`.

_Hint: You can use the same division for `bicycle_rider.bicycle.q` and `bicycle_rider.bicycle.u` as before for `bicycle_def`._

In [ ]:
### BEGIN SOLUTION
b, r = bicycle_rider.bicycle, bicycle_rider.rider
system_br.q_ind = [
    *b.q[:4], *b.q[5:], *b.front_frame.q
]
system_br.q_dep = [
    b.q[4], *r.left_shoulder.q, *r.right_shoulder.q, *r.left_arm.q, *r.right_arm.q
]
system_br.u_ind = [
    b.u[3], *b.u[5:7], *b.front_frame.u
]
system_br.u_dep = [
    *b.u[:3], b.u[4], b.u[7], *r.left_shoulder.u, *r.right_shoulder.u, *r.left_arm.u, *r.right_arm.u
]
### END SOLUTION
system_br.validate_system()

**Exercise**: Form the equations of motion and use `"CRAMER"` as constraint solver.

In [ ]:
### BEGIN SOLUTION
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    eoms = system_br.form_eoms(constraint_solver="CRAMER")
### END SOLUTION
assert system_br.mass_matrix

Congratulations! You have just built your (first) bicycle-rider model.

## Parametrization
While it is possible to manually figure out all parameters and create a dictionary to map all symbols accordingly, BRiM integrates support for the [BicycleParameters](https://github.com/moorepants/BicycleParameters) package.
The `BicycleParameters` package is designed to generate, manipulate, and visualize the parameters of the Whipple-Carvallo bicycle model. The [repository](https://github.com/moorepants/BicycleParameters/tree/master/data) provides several measured parametrizations.
For instructions on measuring your own bicycle and/or rider refer to [Moore's dissertation](https://moorepants.github.io/dissertation/physicalparameters.html).

The code below imports the parametrization library and instantiates a parametrization object, which loads in `"Jason"` on the Batavus `"Browser"` bicycle.

_Note: This cell will download some parametrization data if it is not already present._

In [ ]:
import bicycleparameters as bp
import numpy as np
import os
from utils import download_parametrization_data


data_dir = os.path.join(os.getcwd(), "data")
if not os.path.exists(data_dir):
    download_parametrization_data(data_dir)
bike_params = bp.Bicycle("Browser", pathToData=data_dir)
bike_params.add_rider("Jason", reCalc=True)

To get a mapping of symbols to values based on this parametrization we can use [get_param_values](https://mechmotum.github.io/brim/_autosummary/brim.core.base_classes.BrimBase.html#brim.core.base_classes.BrimBase.get_param_values) as shown below.

In [ ]:
constants_def = bicycle_def.get_param_values(bike_params)
constants_def[g] = 9.81  # Don't forget to specify the gravitational constant.
constants_def

The code below defines a function to quickly retrieve all symbols defined in a model.
Next, we obtain all symbols defined in the bicycle models and if there are still symbols missing in our `constants_def` dictionary.

In [ ]:
missing_symbols = bicycle_def.get_all_symbols().difference(constants_def.keys())
missing_symbols

**Exercise**: Create a `constants_br` dictionary for the `bicycle_rider` model and obtain the missing symbols (`missing_symbols`).

_Note: you may use the same parametrization `bike_params` as before._

In [ ]:
### BEGIN SOLUTION
constants_br = bicycle_rider.get_param_values(bike_params)
missing_symbols = bicycle_rider.get_all_symbols().difference(constants_br.keys())
### END SOLUTION
assert len(constants_br) >= 87
missing_symbols

To quickly see what this symbol is used for we can ask the model to get a description for each:

In [ ]:
for sym in missing_symbols:
    print(f"{sym} - {bicycle_rider.get_description(sym)}")

As these symbols have not been defined by the bicycle parameters data we need to define them manually. However, some of them are inputs, so those we'll specify later on.

In [ ]:
constants_br.update({
    bicycle_rider.bicycle.front_frame.symbols["d9"]: constants_br[bicycle_rider.bicycle.front_frame.symbols["d3"]],
    bicycle_rider.bicycle.front_frame.symbols["k"]: 19.4E3,
    bicycle_rider.bicycle.front_frame.symbols["c"]: 9E3,
    bicycle_rider.seat.symbols["pitch"]: -0.7,
    g: 9.81  # Just in case you did not specify the gravitational constant yet.
})

**Warning**: Above we do forget to take into account the inertia and mass of the legs. A good option would have been computing their contribution and adding that to the rear frame's mass and inertia.

## Simulation Bicycle Model

Now that we have a parametrized model we can simulate it. BRiM does not provide a simulation utility (yet), but a separate module has here been provided implementing a simple simulation utility. The code below imports this utility and initializes a simulation object for the bicycle model.

In [ ]:
import numpy as np
from simulator import Simulator

sim_def = Simulator(system_def)

Before running the actual simulation we need to specify three things:
- `constants`: A dictionary mapping static symbols to values.
- `initial_conditions`: A dictionary mapping generalized coordinates and speeds to initial values.
- `inputs`: A dictionary mapping input symbols to a function taking the time and state as arguments, i.e. `f(t, x) -> float`.

The code below sets each of these for the bicycle model, where an initial guess for the rear frame pitch is provided and the rear wheel is given an initial angular velocity. The disturbance inputs is a sine wave that increases in amplitude over time. For the steer input, we'll use a simple proportional controller that applies a torque proportional to the roll rate to stabilize the bicycle.

In [ ]:
sim_def.constants = constants_def
sim_def.initial_conditions = {
    **{xi: 0.0 for xi in system_def.q.col_join(system_def.u)},
    bicycle_def.q[4]: 0.314,  # Initial guess rear frame pitch.
    bicycle_def.u[5]: -3.0 / constants_def[bicycle_def.rear_wheel.radius],  # Rear wheel angular velocity.
}
roll_rate_idx = len(system_def.q) + system_def.u[:].index(bicycle_def.u[3])
max_roll_rate, max_torque = 0.2, 10
sim_def.inputs = {
    disturbance: lambda t, x: (30 + 30 * t) * np.sin(t * 2 * np.pi),
    steer_torque: lambda t, x: -max_torque * max(-1, min(x[roll_rate_idx] / max_roll_rate, 1)),
}

Now we can initialize the simulator objects, which automatically solves the initial conditions for us.

_Note: the initialization of a simulator object can take a while, so be patient._

In [ ]:
sim_def.initialize()
sim_def.initial_conditions

With the simulator initialized we can run the simulation. The code below runs the simulation for 2.5 seconds. A DAE solver is used with a relative tolerance of 1e-3 and an absolute tolerance of 1e-6.

In [ ]:
sim_def.solve(np.arange(0, 2.5, 0.01), solver="dae", rtol=1e-3, atol=1e-6)
sim_def.t[-1]

## Visualizing the Bicycle Simulation Results

Using the [Plotter](https://mechmotum.github.io/brim/_autosummary/brim.utilities.plotting.Plotter.html) from `BRiM` we can easily visualize the model and create an animation based on the simulation results.

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from scipy.interpolate import CubicSpline
from brim.utilities.plotting import Plotter

# Create some functions to interpolate the results.
x_eval = CubicSpline(sim_def.t, sim_def.x.T)
r_eval = CubicSpline(sim_def.t, [[cf(t, x) for cf in sim_def.inputs.values()]
                                 for t, x in zip(sim_def.t, sim_def.x.T)])
p, p_vals = zip(*sim_def.constants.items())
max_disturbance = r_eval(sim_def.t)[:, tuple(sim_def.inputs.keys()).index(disturbance)].max()

# Plot the initial configuration of the model
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(8, 8))
plotter = Plotter.from_model(ax, bicycle_def)
plotter.add_vector(disturbance * bicycle_def.rear_frame.wheel_hub.axis / max_disturbance,
                   bicycle_def.rear_frame.saddle.point, name="disturbance", color="r")
plotter.lambdify_system((system_def.q[:] + system_def.u[:], sim_def.inputs.keys(), p))
plotter.evaluate_system(x_eval(0.0), r_eval(0.0), p_vals)
plotter.plot()
X, Y = np.meshgrid(np.arange(-1, 10, 0.5), np.arange(-1, 3, 0.5))
ax.plot_wireframe(X, Y, np.zeros_like(X), color="k", alpha=0.3, rstride=1, cstride=1)
ax.invert_zaxis()
ax.invert_yaxis()
ax.set_xlim(X.min(), X.max())
ax.set_ylim(Y.min(), Y.max())
ax.view_init(19, 14)
ax.set_aspect("equal")
ax.axis("off")

def animate(fi):
    """Update the plot for frame i."""
    time = fi / (n_frames - 1) * sim_def.t[-1]
    plotter.evaluate_system(x_eval(time), r_eval(time), p_vals)
    return *plotter.update(),

fps = 30
n_frames = int(fps * sim_def.t[-1])
ani = FuncAnimation(fig, animate, frames=n_frames, blit=False)
display(HTML(ani.to_jshtml(fps=fps)))
plt.close()

## Simulating the Bicycle-Rider Model
Now that we know how to simulate the bicycle model, we can perform a similar simulation for the bicycle-rider model we have created.

**Exercise**: Initiate a simulator object for the bicycle-rider model and assign in to `sim_br`.

In [ ]:
### BEGIN SOLUTION
sim_br = Simulator(system_br)
### END SOLUTION
assert sim_br.system is system_br

**Exercise**: Set the constants, initial conditions, and inputs of `sim_br`. For the inputs try using a similar control for the elbow torques as before for the steer torque.

_Hint: You can access the torques of the load groups from their `symbols` dictionary, e.g. `left_arm_lg.symbols["T"]`._

In [ ]:
### BEGIN SOLUTION
sim_br.constants = constants_br
sim_br.initial_conditions = {
    **{xi: 0.0 for xi in system_br.q.col_join(system_br.u)},
    bicycle_rider.bicycle.q[4]: 0.314,  # Initial guess rear frame pitch.
    bicycle_rider.bicycle.u[5]: -3.0 / constants_br[bicycle_rider.bicycle.rear_wheel.radius],
    bicycle_rider.rider.left_arm.q[0]: 0.7,  # Elbow flexion.
    bicycle_rider.rider.right_arm.q[0]: 0.7,  # Elbow flexion.
    bicycle_rider.rider.left_shoulder.q[0]: 0.5,  # Shoulder flexion.
    bicycle_rider.rider.right_shoulder.q[0]: 0.5,  # Shoulder flexion.
    bicycle_rider.rider.left_shoulder.q[1]: -0.6,  # Shoulder rotation.
    bicycle_rider.rider.left_shoulder.q[1]: -0.6,  # Shoulder rotation.
    bicycle_rider.bicycle.u[3]: 0.3,
}
roll_idx = system_br.q[:].index(bicycle_rider.bicycle.q[3])
roll_rate_idx = len(system_br.q) + system_br.u[:].index(bicycle_rider.bicycle.u[3])
max_roll_rate, max_torque = 0.2, 10
bound = lambda x: max(-1, min(x, 1))
excitation = lambda x:  bound(x[roll_rate_idx] / max_roll_rate)

sim_br.inputs = {
    disturbance: lambda t, x: (30 + 30 * t) * np.sin(t * 2 * np.pi),
    left_arm_lg.symbols["T"]: lambda t, x: -max_torque * excitation(x),
    right_arm_lg.symbols["T"]: lambda t, x: max_torque * excitation(x),
}
### END SOLUTION
assert len(sim_br.constants) == 92
assert len(sim_br.initial_conditions) == 30
assert len(sim_br.inputs) == 3

**Exercise**: Initialize the simulator.

_Hint: If a runtime error from scipy.fsolve is shown, then it was not able to solve the initial conditions. Recommendable is to update your initial conditions to provide a better guess and use `sim_br.solve_initial_conditions()` without reinitializing the simulator object._

_Note: Initializing the bicycle-rider simulator takes longer than initializing just the bicycle._

In [ ]:
### BEGIN SOLUTION
sim_br.initialize()
### END SOLUTION
sim_br.eval_rhs(0, [sim_br.initial_conditions[xi] for xi in system_br.q[:] + system_br.u[:]])

**Exercise**: Peform a simulation over a time-span of 2.5 seconds using a DAE solver.

_Hint: If the simulation fails it can be that the controls are unstable causing the bicycle to fall or that something else is wrong. A simple solution is to use an ODE integrator to figure out the controls: `sim_br.solve((0, 2.5), t_eval=np.arange(0, 2.5, 0.01))`. Also, feel free to reduce the disturbance force to make the control easier._

In [ ]:
### BEGIN SOLUTION
# sim_br.solve(np.arange(0, 2.5, 0.01), solver="dae", rtol=1e-3, atol=1e-6)
sim_br.solve((0, 2.5), t_eval=np.arange(0, 2.5, 0.01))
### END SOLUTION
sim_br.t[-1]

## Visualizing the Bicycle-Rider Simulation

The simulation can be visualized using similar code as below.

**Exercise**: Complete the code below by initializing the [Plotter](https://mechmotum.github.io/brim/_autosummary/brim.utilities.plotting.Plotter.html), optionally adding a vector for the disturbance, and running [lambdify_system](https://mechmotum.github.io/brim/_autosummary/brim.utilities.plotting.Plotter.html#brim.utilities.plotting.Plotter.lambdify_system).

In [ ]:
# Create some functions to interpolate the results.
x_eval = CubicSpline(sim_br.t, sim_br.x.T)
r_eval = CubicSpline(sim_br.t, [[cf(t, x) for cf in sim_br.inputs.values()]
                                 for t, x in zip(sim_br.t, sim_br.x.T)])
p, p_vals = zip(*sim_br.constants.items())
max_disturbance = r_eval(sim_br.t)[:, tuple(sim_br.inputs.keys()).index(disturbance)].max()

# Plot the initial configuration of the model
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(8, 8))
### BEGIN SOLUTION
plotter = Plotter.from_model(ax, bicycle_rider)
plotter.add_vector(disturbance * bicycle_rider.bicycle.rear_frame.wheel_hub.axis / max_disturbance,
                   bicycle_rider.bicycle.rear_frame.saddle.point, name="disturbance", color="r")
plotter.lambdify_system((system_br.q[:] + system_br.u[:], sim_br.inputs.keys(), p))
### END SOLUTION
plotter.evaluate_system(x_eval(0.0), r_eval(0.0), p_vals)
plotter.plot()
X, Y = np.meshgrid(np.arange(-1, 10, 0.5), np.arange(-1, 4, 0.5))
ax.plot_wireframe(X, Y, np.zeros_like(X), color="k", alpha=0.3, rstride=1, cstride=1)
ax.invert_zaxis()
ax.invert_yaxis()
ax.set_xlim(X.min(), X.max())
ax.set_ylim(Y.min(), Y.max())
ax.view_init(19, 14)
ax.set_aspect("equal")
ax.axis("off")

def animate(fi):
    """Update the plot for frame i."""
    time = fi / (n_frames - 1) * sim_br.t[-1]
    plotter.evaluate_system(x_eval(time), r_eval(time), p_vals)
    return *plotter.update(),

fps = 30
n_frames = int(fps * sim_br.t[-1])
ani = FuncAnimation(fig, animate, frames=n_frames, blit=False)
display(HTML(ani.to_jshtml(fps=fps)))
plt.close()

Congratulations! You have completed this tutorial.